In [1]:
#@title # **📔 TensorPrime**#{ vertical-output: true, form-width: "50%", display-mode: "form" }

#@markdown #### ↖️ Click the ▶️ button after deciding on the options below.
#@markdown #### 🔌 Make sure the TPU is enabled under *Runtime→Change runtime type*
#@markdown #### 💡 Keep each notebook **open** in the browser to prevent disconnection. 📌 [the tab](https://support.mozilla.org/en-US/kb/pinned-tabs-keep-favorite-websites-open) or move them to a dedicated window for easy access.
#@markdown #### ℹ️ This notebook uses our [TensorPrime](https://github.com/TPU-Mersenne-Prime-Search/TensorPrime) and our [PrimeNet Python script](https://github.com/tdulcet/Distributed-Computing-Scripts#primenet).
#@markdown #### 📜 Please see the [wiki](TPU-Mersenne-Prime-Search/TensorPrime) for more information.
#@markdown #### 🤷 Optionally, create a GIMPS/PrimeNet account [here](https://www.mersenne.org/update/) and [join](https://www.mersenne.org/jteam/) the “Portland State University” team!

prime_ID = 'Default' #@param ['Default'] {allow-input: true}
computer_name = 'Default' #@param ['Default'] {allow-input: true}
TPU_type_of_work = '150 - First time PRP tests' #@param ['150 - First time PRP tests', '151 - Double-check PRP tests']
computer_number = 'Default (1)' #@param ['Default (1)', '2', '3', '4'] {allow-input: true}
output_type = 'TPU (TensorPrime)' #@param ['TPU (TensorPrime)']
local_time = 'Pacific' #@param ['Pacific', 'Mountain', 'Central', 'Eastern', 'Alaska', 'Hawaii']
debug = 'False' #@param ['False', 'TPU (<Insert TPU program name>)', 'CPU (Prime95)']


#@markdown #### 🐛 The *debug* option outputs TPU (TensorPrime) progress and status, then exits.

In [ ]:
import logging
import time

LOGGER_NAME = "mersenne"
def create_mersenne_logger(name):
    # set delay to true in order to only write on .emit()
    # mode = w for wrte, recreates per runtime. if we want a per runtime file generated, we use time_name to create 
    # a unique name based on the time the file is run
    #
    # mode = a for appending to an extant file, otherwise identical to write
    global LOGGER_NAME
    m_logger = logging.getLogger(LOGGER_NAME, level = "info")
    fh = logging.FileHandler(filename = name, mode = "a", delay = False)
    fh.setFormatter(logging.Formatter("%(asctime)s %(message)s", level = logging.INFO))
    m_logger.addHandler(fh)
    return m_logger

def time_name(name: str = None):
    if name == None:
        name = ".log"
    return time.strftime("%Y%m%dT%H%M%S", time.gmtime()) + "_" + name

m_logger = create_mersenne_logger(time_name("Mersenne.log"))



In [ ]:
import os

tensorprime_path = "/content/drive/My Drive/GIMPS/TensorPrime"

'''Load & cd into gdrive for persistent data'''
global path_dir
if os.path.exists("/content/drive/My Drive"): # create your own notebook with our code
  %cd "/content/drive/My Drive"
else: # use our notebook
  print('Warning: Google Drive is not mounted')
  print('If you were not expecting this, on the far left click the folder icon, the "Mount Drive" folder button, select "CONNECT TO GOOGLE DRIVE" ')
  print('and then re-execute this cell.')
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd "/content/gdrive/My Drive"
  %mkdir -p GIMPS
  %cd GIMPS
if not os.path.exists("/content/drive/My Drive/GIMPS/TensorPrime/"):
  !git clone https://github.com/TPU-Mersenne-Prime-Search/TensorPrime/
%cd TensorPrime
!git pull && git checkout colab-notebook-bootstrap
!python3 main.py -p 127 